# ANN to model Cl

In [ ]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import torch

results_dir = "results/"
cuda0 = torch.device("cuda:0")


class AerofoilLiftSample:
    """
    AerfoilLiftSample
    A single sample of aerofoil data generated from XFoil

    Params:
        file_name: relative file of cpwr export from XFoil

    Attributes:
        feature: 2D tensor of size 1x2x80 of Cp data
        label: 2D tesnor of size 2x2x80 of x,y, data
        airfoil_params: 1D tensor of naca code and simulation paramaters

    """

    def __init__(self, file_name):
        self.file_name = file_name
        aerofoil_data = self.file_name.split("_")
        naca = aerofoil_data[1]
        self.camber = naca[0]
        self.camber_pos = naca[1]
        self.thickness = naca[2:3]
        self.reynolds = aerofoil_data[2]
        self.mach = aerofoil_data[3]
        self.alpha = aerofoil_data[4].split(".")[0]

    def __str__(self):
        return self.file_name

    def feature(self):
        return torch.tensor(
            [
                [self.cp_data()["x"][:80], self.cp_data()["y"][:80]],
                [self.cp_data()["x"][80:], self.cp_data()["y"][80:]],
            ],
            dtype=torch.float32,
            device=cuda0,
        )

    def airfoil_params(self):
        return torch.tensor(
            [
                float(self.camber),
                float(self.camber_pos),
                float(self.thickness),
                float(self.reynolds),
                float(self.mach),
                float(self.alpha),
            ],
            dtype=torch.float32,
            device=cuda0,
        )

    def cp_data(self):
        file = results_dir + self.file_name
        df = pd.read_csv(
            file,
            sep="\s+",
            skiprows=[0, 1, 2],
            names=["x", "y", "Cp"],
            dtype=float,
            on_bad_lines="skip",
        )
        return df

    def cl(self):
        return torch.tensor(
            np.trapz(self.cp_data()["Cp"], self.cp_data()["x"]),
            dtype=torch.float,
            device=cuda0,
        )

    def label(self):
        return torch.tensor(
            [[self.cp_data()["Cp"][:80], self.cp_data()["Cp"][80:]]],
            dtype=torch.float,
            device=cuda0,
        )


class AerofoilLiftDataset(Dataset):
    """
    AerofoilLiftDataset

    Dataset of all samples in results dir
    """

    def __init__(self):
        self.samples = self.load_samples()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return (
            self.samples[item].feature(),
            self.samples[item].airfoil_params(),
            self.samples[item].label(),
        )

    def load_samples(self):
        samples = []
        for file in os.listdir(results_dir):
            if "cpwr" in file:
                samples.append(AerofoilLiftSample(file))
        return samples

In [ ]:
from torch.utils.data import DataLoader, random_split

# Build Dataset
dataset = AerofoilLiftDataset()

# Split dataset for training and validation
train_validate_split = 0.8
train_size = int(train_validate_split * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataLoader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataLoader = DataLoader(test_dataset, batch_size=32)

In [ ]:
from torch.utils.tensorboard import SummaryWriter


class CNNModel(torch.nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.cnn1 = torch.nn.Conv2d(
            in_channels=2, out_channels=2, kernel_size=3, padding=1, device=cuda0
        )
        self.cnn2 = torch.nn.Conv2d(
            in_channels=2, out_channels=1, kernel_size=4, padding=1, device=cuda0
        )
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.2)
        self.l1 = torch.nn.Linear(85, 1024, device=cuda0)
        self.l2 = torch.nn.Linear(1024, 640, device=cuda0)
        self.l3 = torch.nn.Linear(640, 79, device=cuda0)
        self.tcnn1 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=2, padding=1, device=cuda0
        )

    def forward(self, pos_data, airfoil_data):
        x = self.cnn1(pos_data)
        x = self.cnn2(self.relu(x))
        x = torch.cat((self.relu(x).squeeze(), airfoil_data), dim=1)
        x = self.l1(x)
        x = self.l2(self.dropout(self.relu(x)))
        x = self.l3(self.relu(x))
        x = self.tcnn1(x[:, None, None, :])
        return x


model = CNNModel()
loss_fn = torch.nn.L1Loss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random


def plot_results(idx=None):
    """
    Plot Cp profile of aerofoil from test dataset

    Params:
        idx: index of sample in dataset, random if None (default)
    """
    df = pd.DataFrame()
    if not idx:
        idx = random.randint(0, 20)
    test_data = test_dataset[idx]
    test_feature, test_param, test_cp_data = test_data
    test_feature = test_feature.repeat(2, 1, 1, 1)
    test_param = test_param.repeat(2, 1)
    result = model(test_feature, test_param).cpu().detach()
    result_cp = torch.unbind(result, 0)[0].squeeze().cpu().detach().tolist()
    test_cp = torch.unbind(test_cp_data, 0)[0].squeeze().cpu().detach().tolist()
    cords = torch.unbind(test_feature, 0)[0].squeeze().cpu().detach().tolist()
    df["Cp"] = [*result_cp[0], *result_cp[1]]
    df["x"] = [*cords[0][0], *cords[1][0]]
    df["y"] = [*cords[0][1], *cords[1][1]]
    df["valid_Cp"] = [*test_cp[0], *test_cp[1]]
    df.head(300)
    fig, ax1 = plt.subplots()
    ax1.plot(df["x"], df["Cp"], label="NN")
    ax1.plot(df["x"], df["valid_Cp"], label="XFoil")
    ax1.set_ylabel("Cp")
    ax1.set_xlabel("x")
    ax1.legend()
    cp_lim = ax1.get_ylim()
    ax1.set_ylim(cp_lim[1], cp_lim[0])
    ax2 = ax1.twinx()
    ax2.plot(df["x"], df["y"], color="k")
    ax2.set_ylim(-0.4, 0.4)
    ax2.axis("off")
    plt.show()

In [ ]:
from tqdm import tqdm

writer = SummaryWriter()


for epoch in range(500):
    print(f"Epoch: {epoch}")
    training_loss = []
    validation_loss = []
    for train_features, train_params, train_labels in tqdm(train_dataLoader):
        y_pred = model(train_features, train_params)
        loss = loss_fn(y_pred, train_labels)
        optim.zero_grad(set_to_none=True)
        loss.backward()
        optim.step()
        training_loss.append(loss.item())
    writer.add_scalar("training_loss", np.nanmean(training_loss), epoch)
    if epoch % 5 == 0:
        with torch.no_grad():
            for validation_features, validation_params, validation_labels in tqdm(
                test_dataLoader
            ):
                y_pred = model(validation_features, validation_params)
                loss = loss_fn(y_pred, validation_labels)
                validation_loss.append(loss.item())
        writer.add_scalar("validation_loss", np.nanmean(validation_loss), epoch)
        plot_results()

In [ ]:
plot_results()